In [13]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Client:
    def __init__(self, point, cost):
        self.point = point
        self.cost = cost
        
class Office:
    def __init__(self, point, cost, storage):
        self.point = point
        self.cost = cost
        self.storage = storage

In [61]:
## procesamiento de datos
#"./data/fl_1000_2"
def procesarDatos(url):
    sedes = []
    clientes = []
    with open(url) as f:
        i = 1
        for line in f:
            l = line.split()
            if i == 1:
                cant_sedes = int(l[0])
                cant_clientes = int(l[1])
            else:
                if (cant_sedes > 0):
                    sedes.append(Office(Point(float(l[2]), float(l[3])), int(l[0]) , int(l[1]) ))
                else:
                    clientes.append(Client(Point(float(l[1]), float(l[2])), int(l[0])))
                cant_sedes = cant_sedes - 1    
            i = i + 1
    return (sedes, clientes)